# 🚀 JetX Predictor - Modüler Eğitim Orkestrasyonu (v6.0 - T4 GPU)

Bu notebook, JetX modellerini **tek tek** veya **hepsini bir arada** eğitmenize olanak tanır.
Google Colab T4 GPU ortamı için optimize edilmiştir.

### 📋 Özellikler:
- **Parçalı Eğitim:** İstediğiniz modeli seçip sadece onu eğitebilirsiniz.
- **Anında İndirme:** Her eğitim bitiminde model dosyaları otomatik iner.
- **T4 Optimizasyonu:** Büyük batch size ve mixed precision ayarları.

### 🛠️ Nasıl Kullanılır:
1. **Hücre 2 (Kontrol Paneli)**'den eğitmek istediğiniz modelleri `True` yapın.
2. **Hücre 1** ve **Hücre 3**'ü çalıştırarak ortamı hazırlayın.
3. İlgili modelin hücresini çalıştırın.

💾 **Yedekleme:** Dosyalar `/content/drive/MyDrive/JetX_Models_Backup_v6/` klasörüne de yedeklenir.

In [ ]:
# 🛠️ HÜCRE 1: ROBUST ORTAM KURULUMU VE YARDIMCI FONKSİYONLAR

import os
import sys
import subprocess
import time
import shutil
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 JetX Predictor v6.0 - Environment Setup")
print("="*80)

# 1. Google Drive Bağlantısı
try:
    from google.colab import drive
    from google.colab import files
    drive.mount('/content/drive')
    BACKUP_DIR = '/content/drive/MyDrive/JetX_Models_Backup_v6'
    os.makedirs(BACKUP_DIR, exist_ok=True)
    print(f"✅ Google Drive bağlandı. Yedekleme: {BACKUP_DIR}")
except:
    BACKUP_DIR = None
    print("⚠️ Google Drive bağlanamadı. Sadece lokal indirme yapılacak.")

# 2. Paket Kurulumları
print("\n📦 GEREKLİ PAKETLER KURULUYOR...")
install_order = [
    ['numpy', 'pandas', 'scikit-learn', 'joblib', 'scipy'],
    ['tensorflow', 'torch'],
    ['catboost', 'xgboost', 'lightgbm', 'autogluon', 'pytorch-tabnet'],
    ['optuna', 'shap', 'nolds', 'PyWavelets'],
    ['matplotlib', 'seaborn', 'tqdm', 'pyyaml']
]
for packages in install_order:
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q'] + packages, check=False)
    except: pass
print("✅ Paketler hazır.")

# 3. Proje Çekme
PROJECT_ROOT = '/content/jetxpredictor'
if not os.path.exists(PROJECT_ROOT):
    subprocess.run(['git', 'clone', 'https://github.com/onndd/jetxpredictor.git'], check=True)
else:
    # Güncel kodu çek
    os.chdir(PROJECT_ROOT)
    subprocess.run(['git', 'pull'], check=False)

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
print(f"✅ Proje dizini: {os.getcwd()}")

# 4. İNDİRME VE YEDEKLEME FONKSİYONU (KRİTİK)
def backup_and_download(file_path, description="Dosya"):
    """Dosyayı Drive'a yedekler ve tarayıcıdan indirir"""
    if not os.path.exists(file_path):
        print(f"❌ {description} bulunamadı: {file_path}")
        return
    
    # Drive'a kopyala
    if BACKUP_DIR:
        try:
            # Eğer klasörse zip'le
            if os.path.isdir(file_path):
                zip_name = f"{file_path}.zip"
                if not os.path.exists(zip_name):
                     shutil.make_archive(file_path, 'zip', file_path)
                file_to_copy = zip_name
            else:
                file_to_copy = file_path
                
            dest = os.path.join(BACKUP_DIR, os.path.basename(file_to_copy))
            shutil.copy(file_to_copy, dest)
            print(f"💾 Drive'a yedeklendi: {dest}")
        except Exception as e:
            print(f"⚠️ Drive yedekleme hatası: {e}")
    
    # İndir
    try:
        if os.path.isdir(file_path):
            # Klasörse ziplediğimizi indir
            if not os.path.exists(f"{file_path}.zip"):
                shutil.make_archive(file_path, 'zip', file_path)
            files.download(f"{file_path}.zip")
        else:
            files.download(file_path)
        print(f"⬇️ {description} indiriliyor...")
    except Exception as e:
        print(f"⚠️ İndirme hatası: {e}")

In [ ]:
# 🎛️ HÜCRE 2: KONTROL PANELİ (BURAYI DÜZENLEYİN)
# Hangi modeli eğitmek istiyorsanız onu True yapın.

# 🚀 MODEL SEÇİMİ (Tek tek True yapabilirsiniz)
TRAIN_PROGRESSIVE = True     # 1. Ana Model (NN Transformer - En Önemli)
TRAIN_ULTRA = False          # 2. Ultra Aggressive
TRAIN_CATBOOST = False       # 3. CatBoost Ensemble
TRAIN_AUTOGLUON = False      # 4. AutoGluon
TRAIN_TABNET = False         # 5. TabNet
TRAIN_RL_AGENT = False       # 6. RL Agent
TRAIN_META_MODEL = False     # 7. Meta Model

# ⚡ AYARLAR
QUICK_TEST_MODE = False  # Hızlı test için True yapın
MODELS_DIR = 'models'
os.makedirs(MODELS_DIR, exist_ok=True)

# 📊 EĞİTİM KONFIGURASYONU (T4 GPU OPTIMIZE)
TRAINING_CONFIG = {
    'epochs': 5 if QUICK_TEST_MODE else 100,
    'catboost_iterations': 100 if QUICK_TEST_MODE else 1500,
    'autogluon_time': 60 if QUICK_TEST_MODE else 3600,
    # T4 GPU 16GB VRAM olduğu için batch size'ı artırıyoruz
    'batch_size': 16 if QUICK_TEST_MODE else 256 
}

print(f"⚡ QUICK_TEST_MODE: {QUICK_TEST_MODE}")
print(f"🔧 BATCH SIZE: {TRAINING_CONFIG['batch_size']}")
print(f"🎯 SEÇİLEN MODELLER:")
if TRAIN_PROGRESSIVE: print("  - Progressive Transformer")
if TRAIN_ULTRA: print("  - Ultra Aggressive")
if TRAIN_CATBOOST: print("  - CatBoost Ensemble")
if TRAIN_AUTOGLUON: print("  - AutoGluon")
if TRAIN_TABNET: print("  - TabNet")
if TRAIN_RL_AGENT: print("  - RL Agent")
if TRAIN_META_MODEL: print("  - Meta Model")

In [ ]:
# 📊 HÜCRE 3: VERİ HAZIRLIĞI
# Veritabanı kontrolü ve hazırlığı

import sqlite3
import pandas as pd
import numpy as np

print("📊 Veri kontrol ediliyor...")
db_path = 'jetx_data.db'

# Sentetik veri oluşturucu (Eğer gerçek veri yoksa)
if not os.path.exists(db_path):
    print("⚠️ Veritabanı yok. Sentetik veri oluşturuluyor (Eğitim için)...")
    values = np.random.lognormal(0.5, 0.8, 3000)
    values = np.clip(values, 1.0, 100.0)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS jetx_results (id INTEGER PRIMARY KEY AUTOINCREMENT, value REAL, timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)")
    for val in values:
        cursor.execute("INSERT INTO jetx_results (value) VALUES (?)", (val,))
    conn.commit()
    conn.close()
    print(f"✅ {len(values)} adet sentetik veri oluşturuldu.")
else:
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT value FROM jetx_results", conn)
    conn.close()
    print(f"✅ {len(df)} adet veri yüklendi.")

In [ ]:
# 🧠 HÜCRE 4: PROGRESSIVE TRANSFORMER EĞİTİMİ (MONOLITHIC)
# En önemli model. N-Beats, TCN ve Transformer içerir.

if TRAIN_PROGRESSIVE:
    print("\n" + "="*80)
    print("🧠 PROGRESSIVE TRANSFORMER EĞİTİMİ BAŞLIYOR")
    print("="*80)
    
    try:
        subprocess.run([sys.executable, 'notebooks/jetx_PROGRESSIVE_TRAINING.py'], check=True)
        
        # Çıktıları yedekle ve indir
        backup_and_download('models/jetx_progressive_final.h5', 'Progressive Model (Final)')
        
        # Transformer kopyasını da al (sistem bunu arıyor olabilir)
        if os.path.exists('models/jetx_progressive_final.h5'):
            import shutil
            shutil.copy('models/jetx_progressive_final.h5', 'models/jetx_progressive_transformer.h5')
            backup_and_download('models/jetx_progressive_transformer.h5', 'Transformer Model (Copy)')
            
        backup_and_download('models/scaler_progressive.pkl', 'Progressive Scaler')
        backup_and_download('models/model_info.json', 'Model Info')
        
    except Exception as e:
        print(f"❌ HATA: {e}")
else:
    print("⏭️ Progressive eğitimi atlandı.")

In [ ]:
# ⚡ HÜCRE 5: ULTRA AGGRESSIVE MODEL EĞİTİMİ

if TRAIN_ULTRA:
    print("\n⚡ ULTRA AGGRESSIVE MODEL EĞİTİLİYOR...")
    try:
        subprocess.run([sys.executable, 'notebooks/jetx_model_training_ULTRA_AGGRESSIVE.py'], check=True)
        
        backup_and_download('models/jetx_ultra_model.h5', 'Ultra Model')
        backup_and_download('models/scaler_ultra.pkl', 'Ultra Scaler')
        backup_and_download('ultra_model_info.json', 'Ultra Info')
        
    except Exception as e:
        print(f"❌ HATA: {e}")
else:
    print("⏭️ Ultra atlandı.")

In [ ]:
# 🚀 HÜCRE 6: CATBOOST ENSEMBLE EĞİTİMİ

if TRAIN_CATBOOST:
    print("\n🚀 CATBOOST ENSEMBLE EĞİTİLİYOR...")
    try:
        subprocess.run([sys.executable, 'notebooks/jetx_CATBOOST_TRAINING_MULTISCALE.py'], check=True)
        
        # Klasörü zipleyip indir
        backup_and_download('models/catboost_multiscale', 'CatBoost Models (Folder)')
        
    except Exception as e:
        print(f"❌ HATA: {e}")
else:
    print("⏭️ CatBoost atlandı.")

In [ ]:
# 🤖 HÜCRE 7: AUTOGLUON EĞİTİMİ

if TRAIN_AUTOGLUON:
    print("\n🤖 AUTOGLUON EĞİTİLİYOR...")
    try:
        # AutoGluon kodunu burada simüle ediyoruz (Gerekirse script'e taşıyın)
        from utils.autogluon_predictor import AutoGluonPredictor
        from utils.database import DatabaseManager
        from category_definitions import FeatureEngineering
        
        db = DatabaseManager('jetx_data.db')
        data = db.get_all_results()
        
        if len(data) > 500:
            X, y = [], []
            limit = min(len(data), 3000)
            for i in range(500, limit):
                 hist = data[i-500:i]
                 target = data[i]
                 feats = FeatureEngineering.extract_all_features(hist)
                 X.append(list(feats.values()))
                 y.append(1 if target >= 1.5 else 0)
            
            X_df = pd.DataFrame(X)
            y_series = pd.Series(y)
            
            print(f"🚀 AutoGluon eğitiliyor (Süre limiti: {TRAINING_CONFIG['autogluon_time']}s)...")
            predictor = AutoGluonPredictor()
            predictor.train(X_df, y_series, time_limit=TRAINING_CONFIG['autogluon_time'])
            
            # İndir
            backup_and_download('models/autogluon_model', 'AutoGluon Model (Folder)')
            backup_and_download('models/autogluon_scaler.pkl', 'AutoGluon Scaler')
        else:
            print("⚠️ Yetersiz veri.")
            
    except Exception as e:
        print(f"❌ HATA: {e}")
else:
    print("⏭️ AutoGluon atlandı.")

In [ ]:
# 🎯 HÜCRE 8: TABNET EĞİTİMİ

if TRAIN_TABNET:
    print("\n🎯 TABNET EĞİTİLİYOR...")
    # TabNet eğitim kodunu buraya entegre edin veya utils/tabnet_predictor.py kullanın
    # Basitlik için placeholder, script varsa subprocess ile çağırın
    print("ℹ️ TabNet eğitimi utils üzerinden çalıştırılabilir.")
    # backup_and_download('models/tabnet_high_x.pkl', 'TabNet Model')
else:
    print("⏭️ TabNet atlandı.")

In [ ]:
# 🤖 HÜCRE 9: RL AGENT EĞİTİMİ

if TRAIN_RL_AGENT:
    print("\n🤖 RL AGENT EĞİTİLİYOR...")
    try:
        subprocess.run([sys.executable, 'notebooks/train_rl_agent.py'], check=True)
        backup_and_download('models/rl_agent_model.h5', 'RL Model')
        backup_and_download('models/rl_agent_info.json', 'RL Info')
    except Exception as e:
        print(f"❌ HATA: {e}")
else:
    print("⏭️ RL Agent atlandı.")

In [ ]:
# 🏆 HÜCRE 10: META-MODEL EĞİTİMİ

if TRAIN_META_MODEL:
    print("\n🏆 META-MODEL EĞİTİLİYOR...")
    try:
        subprocess.run([sys.executable, 'notebooks/TRAIN_META_MODEL.py'], check=True)
        backup_and_download('models/meta_model.json', 'Meta Model')
        backup_and_download('models/meta_model_info.json', 'Meta Info')
    except Exception as e:
        print(f"❌ HATA: {e}")
else:
    print("⏭️ Meta-Model atlandı.")

In [ ]:
# 📦 HÜCRE 11: TOPLU PAKETLEME VE İNDİRME (Opsiyonel)
# Eğer yukarıda tek tek indirdiyseniz buna gerek kalmayabilir

print("\n" + "="*80)
print("📦 TOPLU PAKETLEME VE İNDİRME")
print("="*80)

import shutil
import zipfile

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'JetX_FULL_ARMY_v6.0_{timestamp}.zip'

# Dosyaları topla
files_to_zip = [
    'models/jetx_progressive_transformer.h5',
    'models/scaler_progressive.pkl',
    'models/jetx_ultra_model.h5',
    'models/scaler_ultra.pkl',
    'models/meta_model.json',
    'models/meta_model_info.json',
    'models/rl_agent_model.h5',
    'models/rl_agent_info.json',
    'models/autogluon_model', # Klasör
    'models/autogluon_scaler.pkl',
    'models/tabnet_high_x.pkl',
    'models/tabnet_scaler.pkl',
    'models/catboost_multiscale' # Klasör
]

print("Dosyalar sıkıştırılıyor...")
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for item in files_to_zip:
        if os.path.exists(item):
            if os.path.isdir(item):
                for root, dirs, files in os.walk(item):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, os.path.join(item, '..'))
                        zipf.write(file_path, arcname)
            else:
                zipf.write(item, os.path.basename(item))
            print(f"  Adding: {item}")
        else:
            print(f"  ⚠️ Bulunamadı: {item}")

backup_and_download(zip_filename, 'TOPLU ZIP DOSYASI')

print("\n🎉 TÜM İŞLEMLER TAMAMLANDI!")